# Lesson 1: Advanced RAG Pipeline

In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [3]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data").load_data()
#    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]


In [4]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

87 

<class 'llama_index.schema.Document'>
Doc ID: 76fc43f9-2cc0-418e-b8f0-73ab2efbfd70
Text: 事務連絡  平成２５年４月 １５日    各都道府県衛生主管部（局）薬務主管課 御中       厚生労働省医薬食品局審査管理課
細胞・組織加工医薬品等の製造に関連するものに係る原薬等登録原簿 登録申請書及びその申請書に添付すべき資料の作成要領に関する Q&A について
「細胞・組織加工医薬品等の製造に関連するものに係る原薬等登録原簿、 登 録申請書及びその申請書に添付すべき資料の作成要領」
については、平成２５ 年３月８日付け事務連絡により示したところですが、今般、質疑応答集を別添のとおりまとめましたので、貴管内関係者に対し周知
方よろしく御配慮をお願いします。


## Basic RAG pipeline

In [5]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [6]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
query_engine = index.as_query_engine()

In [8]:
response = query_engine.query(
    "MFとはなんですか?"
)
print(str(response))

MFは、登録事項の変更や承継に関連する医薬品のマスターファイル（Master File）の略称です。


## Evaluation setup using TruLens

In [14]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

MFとはなんですか。日本語で答えてください。
MFの申請書類はなんですか。日本語で答えてください。
USPに登録がる原薬をMFに登録する際の注意点はなんですか。日本語で答えてください。
MFの仕組みを教えてください。日本語で答えてください。


In [ ]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [12]:
print(eval_questions)

[]


In [15]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [16]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [17]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [18]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [19]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Answer Relevance_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_19a60c2e4f59a43b2b47a2b9920bbc36,"""MF\u3068\u306f\u306a\u3093\u3067\u3059\u304b\...","""MF\u306f\u300c\u539f\u85ac\u7b49\u767b\u9332\...",-,"{""record_id"": ""record_hash_19a60c2e4f59a43b2b4...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-20T22:41:07.178819"", ""...",2024-06-20T22:41:08.317577,1.0,0.45,"[{'args': {'prompt': 'MFとはなんですか。日本語で答えてください。',...","[{'args': {'prompt': 'MFとはなんですか。日本語で答えてください。',...",1,2074,0.003122
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_75405ab6fa18d38f3eb5a7b478bf8728,"""MF\u306e\u7533\u8acb\u66f8\u985e\u306f\u306a\...","""MF\u306e\u7533\u8acb\u66f8\u985e\u306f\u3001\...",-,"{""record_id"": ""record_hash_75405ab6fa18d38f3eb...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-20T22:41:08.498149"", ""...",2024-06-20T22:41:10.207188,1.0,0.50,[{'args': {'prompt': 'MFの申請書類はなんですか。日本語で答えてくださ...,[{'args': {'prompt': 'MFの申請書類はなんですか。日本語で答えてくださ...,1,1940,0.002938
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_aba8da3dc93433af963f7478c748c9d4,"""USP\u306b\u767b\u9332\u304c\u308b\u539f\u85ac...","""\u539f\u85ac\u306e\u6b8b\u7559\u6eb6\u5a92\u3...",-,"{""record_id"": ""record_hash_aba8da3dc93433af963...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-20T22:41:10.374231"", ""...",2024-06-20T22:41:14.453109,1.0,1.00,[{'args': {'prompt': 'USPに登録がる原薬をMFに登録する際の注意点は...,[{'args': {'prompt': 'USPに登録がる原薬をMFに登録する際の注意点は...,4,2235,0.003453
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_4055214a132a80ad4bd223060703a338,"""MF\u306e\u4ed5\u7d44\u307f\u3092\u6559\u3048\...","""MF\u767b\u9332\u8005\u306f\u3001\u533b\u85ac\...",-,"{""record_id"": ""record_hash_4055214a132a80ad4bd...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2024-06-20T22:41:14.616523"", ""...",2024-06-20T22:41:18.176021,NaN,NaN,NaN,NaN,3,2023,0.003138


In [20]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at https://s172-31-2-34p15913.lab-aws-production.deeplearning.ai/ .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [35]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [36]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [37]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [38]:
window_response = sentence_window_engine.query(
    "MFとはなんですか。日本語で答えてください。"
)
print(str(window_response))

MFは「原薬等登録原簿」の略称です。


In [39]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [40]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fdd786ee0b0 is calling an instrumented method <function BaseQueryEngine.query at 0x7fdef3f2eef0>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fdd884af610) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fdd786ee0b0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7fdeef34eb90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fdd884af610) using this function.
A new object of type <class 'llama_index.indices.vector_store.retrievers.retriever.VectorIndexRetriever'> at 0x7fdd786e8910 is calling an instrumented method <function BaseRetriever.retrieve at 0x7fdef3f2e290>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other obj

MFとはなんですか。日本語で答えてください。
MFは「原薬等登録原簿」の略称です。


openai request failed <class 'openai.BadRequestError'>=Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 33963 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}. Retries remaining=3.
openai request failed <class 'openai.BadRequestError'>=Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 33692 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}. Retries remaining=3.
openai request failed <class 'openai.BadRequestError'>=Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 33963 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'message

MFの申請書類はなんですか。日本語で答えてください。
MFの申請書類は、医薬品に関するMF登録に使用する電子的な様式についての記載事項と、登録申請書に添付する資料（登録データ）に関する様式があります。


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fdd786ee140 is calling an instrumented method <function Refine.get_response at 0x7fdef2970ee0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fdd884af550) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7fdd786ee0b0 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7fdeef34eb90>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7fdd884af610) using this function.


USPに登録がる原薬をMFに登録する際の注意点はなんですか。日本語で答えてください。
原薬をMFに登録する際の注意点は、製造方法、製造工程管理、品質管理試験、規格及び試験方法、安定性試験、非臨床試験（主として新添加剤）に関する情報を登録することです。


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7fdd786ee140 is calling an instrumented method <function Refine.get_response at 0x7fdef2970ee0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7fdd884af550) using this function.


MFの仕組みを教えてください。日本語で答えてください。
MFは、製造所の名称や製造方法、品質管理試験、規格及び試験方法、安定性試験などの情報を登録することができるシステムです。また、製造工程中の細菌、真菌、ウイルスの不活化／除去処理方法や規格及び試験方法に関する情報も含まれます。MFを利用する際には、登録者と利用者間で情報の取り扱いについて十分な協議が必要とされています.


In [41]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.325,1.0,1.0,33.75,0.055591


In [42]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-31-2-34p15913.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

openai request failed <class 'openai.BadRequestError'>=Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 33969 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}. Retries remaining=3.
openai request failed <class 'openai.BadRequestError'>=Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 33842 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}. Retries remaining=3.
openai request failed <class 'openai.BadRequestError'>=Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, your messages resulted in 33969 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'message

### 2. Auto-merging retrieval

In [43]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [44]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [45]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: 89e4d1be-5eae-4f57-92aa-f042b7aff6d5.
> Parent node text: 21http://www.pmda.go.jp/int -activities/int -harmony/ich/0035.htmlICHM4Q第２部（モジュール２）：品質に関する概括資料、第
...

> Merging 1 nodes into parent node.
> Parent node id: f6a451a8-fd4e-456c-9e98-8992ac433703.
> Parent node text: 21http://www.pmda.go.jp/int -activities/int -harmony/ich/0035.htmlICHM4Q第２部（モジュール２）：品質に関する概括資料、第
...

You can build a portfolio of AI projects by focusing on various aspects such as project structure, general characteristics, manufacturing details, manufacturers involved, manufacturing methods, process control, and related substances. It is important to follow the specified format and guidelines provided for creating the necessary documentation for each project.


In [46]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [47]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

MFとはなんですか。日本語で答えてください。
MFは「薬事法」で規定された医薬品の製造において使用される原薬、中間体、製剤原料の品質及び安全性を確認するための登録制度です。
MFの申請書類はなんですか。日本語で答えてください。
MFの申請書類は、登録申請書と登録データです。
USPに登録がる原薬をMFに登録する際の注意点はなんですか。日本語で答えてください。
原薬、中間体及び製剤原料（バルクのうち特殊な剤型等）については、品質及び安全性が従来の規格及び試験方法においても確立されているものと考えられており、当面、MFを利用することは差し控えられたい。
MFの仕組みを教えてください。日本語で答えてください。
MFは、日本薬局方に収載された医薬品の承認審査において使用される登録制度です。人や動物に共通に使用される医薬品の場合、MFを利用する際は、機構にMF登録を行う必要があります。MF登録番号が付与され、その番号を用いて医薬品や医療機器の取扱いに関する情報が管理されます。


In [48]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.838889,0.6,1.0,4.0,0.000732


In [49]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-31-2-34p15913.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>